# Initialization

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col

# import trimming function
from script.silver.silver_utils import trim_string_columns

# Read Bronze table

In [0]:
df = spark.table("workspace.bronze.erp_px_cat_g1v2")

# Silver Transformations

## Trimming

In [0]:
df = trim_string_columns(df)

## Normalize Maintenance Flag to Boolean

In [0]:
df = df.withColumn(
  "maintenance",
  F.when(F.upper(col("maintenance")) == "YES", F.lit(True))
   .when(F.upper(col("maintenance")) == "NO", F.lit(False))
   .otherwise(None)
  )

## Renaming Columns

In [0]:
rename_map = {
    "id": "category_id",
    "cat": "category",
    "subcat": "subcategory",
    "maintenance": "maintenance_flag"
}

for old_name, new_name in rename_map.items():
    df = df.withColumnRenamed(old_name, new_name)

## Sanity checks of dataframe

In [0]:
df.limit(10).display()

In [0]:
display(df.select("maintenance_flag").distinct())

# Writing Silver Table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_product_category")

## Sanity checks of silver table

In [0]:
%sql
SELECT *
FROM workspace.silver.erp_product_category
LIMIT 10;